# Application Demonstration

The Application has two main functionalities that are demonstrated in this notebook.

**Functionality 1: Publication based RAG and Query Answering**

The RAG pipeline allows user querying of the publication base (Zotero collection) in natural language, thus enabling retrieving information no matter where it is written and even synthesizing know[ledge]

LLM-based answers are always grounded and relevant claims are supported by sources (publication title and section) which are provided to the user.

This functionality encapsulates the following steps and modules:
- pdfProcessing
    - Extracting text/metadata from PDFs
    - Preparing and chunking content for populating the Vector DB
- Vector DB and Embedding models
    - Vector embeddings are computed for the paper chunks (e.g. pretrained ModernBert embedder)
    - Vector embeddings are stored in the vector DB with relevant metadata
    - Enables similarity search for most relevant chunks given a user query
- LLM
    - LLM configuration
    - Prompt building; User and System prompts are constructed, retrieved chunks are passed to the chosen LLM (e.g. Mistral nemo)
    - User query is answered based on retrieved knowledge

*On top of the functionality demonstration, a structured evaluation is performed with several user queries of different difficulties.*

**Functionality 2: External paper search**

If a user finds that relevant information is not covered by the current publication base (Zotero collection), this functionality allows them to retrieve external papers via the SemanticScholar API. This ensures the system can expand its knowledge base dynamically by either actively searching for new queries or recommending papers similar to the ones currently being discussed.

This functionality encapsulates the following steps and modules:
- **Semantic Scholar Integration**
    - **Smart Search Strategy**: A multi-stage retrieval mechanism handles queries. It first attempts a standard paper search, if no results are found, it falls back to searching text snippets, extracting titles from those snippets.
- **RAG Integration & Fallback**
    - **Context Extension**: The RAG pipeline supports a `search_for_new_context` flag. When enabled, if the local vector DB fails to provide sufficient context, the system triggers the online search synchronously to recommend papers tha could potentially cover the user query.

**Disclaimer**:\
These paper recommendations are generated directly by Semantic Scholar’s engine. Because their matching algorithm is proprietary and a 'black box' to us, our project timeline didn't allow for exhaustive testing of their results, please view these as intelligent leads rather than definitive answers. We strongly encourage you to review the papers thoroughly yourself to ensure they fit your work.


***USAGE NOTES:***
- For the first run, set CLEAR_DB_ON_RUN = True to populate VectorDB.
- The outputs of the query demonstrations (Chapters 1 to 4 for functionality 1) are written to the outputs/application_demo folder in case the notebook outputs are difficult to read.

# Functionality 1: Publication based RAG and Query Answering

## 1. Setup & Initialization

In [1]:
import asyncio
import os
import sys
from pathlib import Path

import nest_asyncio
from dotenv import load_dotenv

# Change to parent directory for config.yaml access
parent_dir = Path.cwd().parent
os.chdir(parent_dir)
sys.path.insert(0, str(parent_dir))

from pdfProcessing.docling_PDF_processor import DoclingPDFProcessor
from pdfProcessing.chunking import create_chunks_from_sections
from zotero_integration.zotero_client import ZoteroClient
from backend.services.embedder import EmbeddingService
from backend.services.vector_db import VectorDBService
from backend.services.rag_answer_service import ChromaRagRetriever
from backend.services.recommendation import SemanticScholarService
from backend.services.rag_evaluator import EnhancedRAGEvaluator
from backend.utils import query_rag, ingest_pdf, load_eval_dataset, show_llm_prompt, log_retrieval_results
from llmAG.rag.pipeline import RagPipeline
from llmAG.llm import build_llm

import pandas as pd

# Initializing init env files
load_dotenv()
# Allowing nested event loops
nest_asyncio.apply()
print(f"Working directory: {os.getcwd()}")

2026-01-21 15:37:15,158 - INFO - PyTorch version 2.9.1+cu130 available.


Working directory: D:\Dokumente\Studium\MSc\WS2526\GenAi\LiteraturAssistent\GenAI


In [2]:
# Configuration
EMBEDDER_TYPE = "bert"  # "bert" or "qwen"
CHROMA_PATH = "./backend/chroma_db"
MAX_CHUNK_SIZE = 2500
OVERLAP_SIZE = 200
TOP_K_RETRIEVAL = 5
CLEAR_DB_ON_RUN = True  # Set to True to clear DB and re-ingest all PDFs

# Output directory for full chunk outputs
OUTPUT_DIR = Path("outputs/application_demo")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

os.environ["OLLAMA_BASE_URL"] = "http://localhost:11434"

# Initialize services
print("Initializing Zotero metadata loader...")
try:
    zotero_loader = ZoteroClient(
        library_id=int(os.getenv("ZOTERO_LIBRARY_ID")),
        api_key=os.getenv("ZOTERO_API_KEY"),
    )
    print(f"Zotero metadata loaded")
except Exception as e:
    print(f"Warning: Zotero metadata not available: {e}")
    zotero_loader = None

print("Initializing PDF processor...")
processor = DoclingPDFProcessor()

print("Initializing Semantic Scholar connection...")
rec_service = SemanticScholarService(
    api_key=os.getenv("SEMANTIC_SCHOLAR_API_KEY")
)

print("Initializing embedding service...")
embed_service = EmbeddingService()
embedder = embed_service.load_model(EMBEDDER_TYPE)

print("Initializing ChromaDB...")
db_service = VectorDBService(
    db_path=CHROMA_PATH,
    collection_names={
        "bert": "scientific_papers_bert",
        "qwen": "scientific_papers_qwen"
    }
)

print("Initializing LLM (Ollama mistral-nemo)...")
try:
    llm = build_llm(model="mistral-nemo", temperature=0.1)
    print("LLM initialized")
except Exception as e:
    print(f"Error: LLM initialization failed: {e}")
    print("  Make sure Ollama app is running")
    llm = None

Initializing Zotero metadata loader...
Zotero metadata loaded
Initializing PDF processor...
Initializing Docling Converter...
CUDA detected. Using GPU for PDF Processing.
Initializing Semantic Scholar connection...
Initializing embedding service...
Loading Model Key: bert...
Loading Alibaba-NLP/gte-modernbert-base on cuda...


2026-01-21 15:37:17,572 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Initializing ChromaDB...
Initializing LLM (Ollama mistral-nemo)...
LLM initialized


## 2. Ingest Pipeline

In [3]:
# Check database status
collection = db_service.get_collection(EMBEDDER_TYPE)
chunk_count = collection.count()

print(f"Database status (model: {EMBEDDER_TYPE})")
print(f"  Chunks in database: {chunk_count}")
print(f"  CLEAR_DB_ON_RUN: {CLEAR_DB_ON_RUN}")

if CLEAR_DB_ON_RUN and chunk_count > 0:
    print(f"  Clearing existing {chunk_count} chunks...")
    all_ids = collection.get()['ids']
    if all_ids:
        collection.delete(ids=all_ids)
    print("  Database cleared")

Database status (model: bert)
  Chunks in database: 0
  CLEAR_DB_ON_RUN: True


In [4]:
# Conditional ingestion
pdf_dir = Path.cwd() / "data" / "testPDFs"
pdf_files = list(pdf_dir.glob("*.pdf"))
print(f"Found {len(pdf_files)} PDFs in {pdf_dir}")

collection = db_service.get_collection(EMBEDDER_TYPE)
chunk_count = collection.count()

if chunk_count == 0 or CLEAR_DB_ON_RUN:
    print(f"\nIngesting {len(pdf_files)} PDFs...")
    total_chunks = 0
    for i, pdf in enumerate(pdf_files):
        print(f"[{i + 1}/{len(pdf_files)}]", end="")

        chunks = ingest_pdf(
            pdf_path=pdf,
            processor=processor,
            db_service=db_service,
            embedder=embedder,
            create_chunks_func=create_chunks_from_sections,
            model_key=EMBEDDER_TYPE,
            zotero_loader=zotero_loader,
            max_chunk_size=MAX_CHUNK_SIZE,
            overlap_size=OVERLAP_SIZE
        )

        total_chunks += chunks
    print(f"\nIngestion complete: {total_chunks} chunks from {len(pdf_files)} PDFs")
else:
    print(f"Skipping ingestion ({chunk_count} chunks already in database)")

Found 14 PDFs in D:\Dokumente\Studium\MSc\WS2526\GenAi\LiteraturAssistent\GenAI\data\testPDFs

Ingesting 14 PDFs...
[1/14]
Processing: Kandel et al. - 2023 - Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy.pdf


2026-01-21 15:37:18,243 - INFO - HTTP Request: GET https://api.zotero.org/users/19245007/collections?format=json&limit=100&locale=en-US "HTTP/1.1 200 OK"
2026-01-21 15:37:19,507 - INFO - HTTP Request: GET https://api.zotero.org/users/19245007/collections/Q4WQWNVV/items?format=json&limit=100&locale=en-US "HTTP/1.1 200 OK"
2026-01-21 15:37:19,511 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-21 15:37:19,561 - INFO - Going to convert document batch...
2026-01-21 15:37:19,562 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 1064fff70b16649e2a9cc84da931292b
2026-01-21 15:37:19,597 - INFO - Loading plugin 'docling_defaults'
2026-01-21 15:37:19,599 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-21 15:37:19,635 - INFO - Loading plugin 'docling_defaults'
2026-01-21 15:37:19,639 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']


Cached 24 items from Zotero API.
  Using Zotero metadata: 'Demonstration of an AI-driven workflow for autonom...'


2026-01-21 15:37:19,865 - INFO - Accelerator device: 'cuda:0'
[INFO] 2026-01-21 15:37:19,878 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-01-21 15:37:19,884 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\tnkru\anaconda3\envs\GenAI\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-01-21 15:37:19,885 [RapidOCR] main.py:53: Using C:\Users\tnkru\anaconda3\envs\GenAI\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-01-21 15:37:19,932 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-01-21 15:37:19,933 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\tnkru\anaconda3\envs\GenAI\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-01-21 15:37:19,934 [RapidOCR] main.py:53: Using C:\Users\tnkru\anaconda3\envs\GenAI\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-01-21 15:37:19,962 [RapidOCR] base.py:22: Using engi

  Extracted 21 sections
  Created 22 chunks


...

  Extracted 28 sections
  Created 35 chunks


2026-01-21 15:40:26,190 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-21 15:40:26,201 - INFO - Going to convert document batch...
2026-01-21 15:40:26,202 - INFO - Processing document Zhang et al. - 2024 - Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learni.pdf


  Ingested 35 chunks
[14/14]
Processing: Zhang et al. - 2024 - Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learni.pdf
  Using Zotero metadata: 'Precision autofocus in optical microscopy with liq...'


2026-01-21 15:40:34,793 - INFO - Finished converting document Zhang et al. - 2024 - Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learni.pdf in 8.61 sec.


  Extracted 21 sections
  Created 28 chunks
  Ingested 28 chunks

Ingestion complete: 537 chunks from 14 PDFs


## 3. RAG Pipeline Initialization

In [5]:
# Initialize RAG components
retriever = ChromaRagRetriever(
    embed_service=embed_service,
    db_service=db_service,
    model_name=EMBEDDER_TYPE
)

rag_pipeline = RagPipeline(
    retriever=retriever,
    model="mistral-nemo",
    temperature=0.1
)
print("RAG pipeline initialized")


RAG pipeline initialized


## 4. RAG Pipeline Demonstration

Three example queries, one from each evaluation difficulty tier.

### Tier 1: Direct Factual Question

In [6]:
# Tier 1 Query: Direct factual retrieval
query_tier1 = "What physical quantity is the controller changing (the actuator variable) in the liquid-lens autofocus setup?"

print(f"QUERY (Tier 1): {query_tier1}\n")
print("=" * 80)
print("RETRIEVAL RESULTS")
print("=" * 80)

# Retrieve chunks
query_embedding = embedder.encode([query_tier1])[0]
results = db_service.query(
    model_key=EMBEDDER_TYPE,
    query_embedding=query_embedding.tolist(),
    n_results=TOP_K_RETRIEVAL
)

# Collect full output for file
log_retrieval_results(
    results=results,
    query=query_tier1,
    output_file=OUTPUT_DIR / "tier1_retrieval.txt"
)

QUERY (Tier 1): What physical quantity is the controller changing (the actuator variable) in the liquid-lens autofocus setup?

RETRIEVAL RESULTS

Rank 1 | Distance: 0.2860
ID:      Zhang_et_al.___2024___Precision_autofocus_in_optical_microscopy_with_liquid_lenses_controlled_by_deep_reinforcement_learni.pdf#Introduction_part5
Section: Introduction
Paper:   Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning
Authors: Jing Zhang, Yong-feng Fu, Hao Shen, Quan Liu, Li-ning Sun, Li-guo Chen

Content (1250 chars):
--------------------------------------------------------------------------------
In addition, the integration of software algorithms and simple hardware enables end-to-end optical microscope autofocusing, reducing system complexity and cost. Fast Response: The combination of liquid lenses with millisecond focusing speeds and intelligent focusing algorithms enables the rapid autofocusing of optical microscopes. Robustness: The utiliz

"QUERY: What physical quantity is the controller changing (the actuator variable) in the liquid-lens autofocus setup?\n\n================================================================================\nRETRIEVAL RESULTS\n================================================================================\n\n\n================================================================================\nRank 1 | Distance: 0.2860\n================================================================================\nID:      Zhang_et_al.___2024___Precision_autofocus_in_optical_microscopy_with_liquid_lenses_controlled_by_deep_reinforcement_learni.pdf#Introduction_part5\nSection: Introduction\nPaper:   Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning\nAuthors: Jing Zhang, Yong-feng Fu, Hao Shen, Quan Liu, Li-ning Sun, Li-guo Chen\n\nContent (1250 chars):\n--------------------------------------------------------------------------------\nIn addition, the inte

In [7]:
# Show exact prompt sent to LLM
show_llm_prompt(
    rag_pipeline=rag_pipeline,
    retriever=retriever,
    question=query_tier1,
    top_k=TOP_K_RETRIEVAL
)

EXACT PROMPT SENT TO LLM
Template: answer | Retrieved chunks: 5 | Context: 9950 chars


MESSAGE 1: SYSTEM

You are a RAG assistant answering questions about scientific PDFs using only the provided context.
Use the context as the sole source of truth. Do not guess or use prior knowledge.
Answer with factual statements supported by the context.
Every factual claim must include an inline citation formatted as [Title | Section] placed immediately after the clause it supports.
Citations must use titles and section labels exactly as they appear in the context headers; do not invent, shorten, or paraphrase them.
If only part of the question is supported, answer only that part and state that the remaining parts are not in the provided context; do not ask to search online.
If you cannot answer with exact [Title | Section] citations from the context, respond exactly with: "I do not know based on the provided context because the retrieved sections do not mention this. Would you like me to find re

In [8]:
# Generate LLM answer
response_tier1 = rag_pipeline.run(query_tier1, k=TOP_K_RETRIEVAL, include_sources=True)

print("=" * 80)
print("LLM ANSWER")
print("=" * 80 + "\n")
print(response_tier1.answer)

print("\n" + "=" * 80)
print(f"SOURCES ({len(response_tier1.sources)} documents)")
print("=" * 80)
for i, source in enumerate(response_tier1.sources):
    print(f"\n[{i + 1}] {source.metadata.get('title', 'Unknown')}")
    print(f"    Section: {source.metadata.get('section', 'N/A')}")

2026-01-21 15:40:41,234 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


LLM ANSWER

The controller changes the voltage applied to the liquid lens [Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning | Effect of actions on autofocus performance].

SOURCES (5 documents)

[1] Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning
    Section: Introduction

[2] Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning
    Section: Introduction

[3] Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning
    Section: Effect of actions on autofocus performance

[4] Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning
    Section: Introduction

[5] AutoFocus: AI-driven alignment of nanofocusing X-ray mirror systems
    Section: 5.3 Challenges and Considerations


**Comment**:
- The LLM answer is correct; the voltage applied is indeed the variable the controller adjusts.
- The answer is based on the relevant context.
- The chunks stem from the correct paper without mentioning it explicitly.
- The correct chunks were retrieved, namely chunks 2 (Introduction) and 3 (Effect of actions on autofocus performance).

### Tier 2: Multi-detail Question

In [9]:
# Tier 2 Query: Requires extracting multiple related details
query_tier2 = "List the reward hyperparameters (e.g., alpha, beta, mu, delta) for DRL autofocus and what each incentivizes."

print(f"QUERY (Tier 2): {query_tier2}\n")
print("=" * 80)
print("RETRIEVAL RESULTS")
print("=" * 80)

# Retrieve chunks
query_embedding = embedder.encode([query_tier2])[0]
results = db_service.query(
    model_key=EMBEDDER_TYPE,
    query_embedding=query_embedding.tolist(),
    n_results=TOP_K_RETRIEVAL
)

log_retrieval_results(
    results=results,
    query=query_tier2,
    output_file=OUTPUT_DIR / "tier2_retrieval.txt"
)

QUERY (Tier 2): List the reward hyperparameters (e.g., alpha, beta, mu, delta) for DRL autofocus and what each incentivizes.

RETRIEVAL RESULTS

Rank 1 | Distance: 0.2574
ID:      Zhang_et_al.___2024___Precision_autofocus_in_optical_microscopy_with_liquid_lenses_controlled_by_deep_reinforcement_learni.pdf#Reward_function_part1
Section: Reward function
Paper:   Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning
Authors: Jing Zhang, Yong-feng Fu, Hao Shen, Quan Liu, Li-ning Sun, Li-guo Chen

Content (721 chars):
--------------------------------------------------------------------------------
The last term δ is an additional reward component aimed at enhancing the discriminative ability of the reward function by setting relatively large positive and negative rewards for the clearest and least clear images, respectively, thereby further reducing the focusing steps. Since achieving clear imaging, reducing the time to focus, and stopping au

"QUERY: List the reward hyperparameters (e.g., alpha, beta, mu, delta) for DRL autofocus and what each incentivizes.\n\n================================================================================\nRETRIEVAL RESULTS\n================================================================================\n\n\n================================================================================\nRank 1 | Distance: 0.2574\n================================================================================\nID:      Zhang_et_al.___2024___Precision_autofocus_in_optical_microscopy_with_liquid_lenses_controlled_by_deep_reinforcement_learni.pdf#Reward_function_part1\nSection: Reward function\nPaper:   Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning\nAuthors: Jing Zhang, Yong-feng Fu, Hao Shen, Quan Liu, Li-ning Sun, Li-guo Chen\n\nContent (721 chars):\n--------------------------------------------------------------------------------\nThe last term δ i

In [10]:
# Show exact prompt sent to LLM
show_llm_prompt(
    rag_pipeline=rag_pipeline,
    retriever=retriever,
    question=query_tier2,
    top_k=TOP_K_RETRIEVAL
)

EXACT PROMPT SENT TO LLM
Template: answer | Retrieved chunks: 5 | Context: 7509 chars


MESSAGE 1: SYSTEM

You are a RAG assistant answering questions about scientific PDFs using only the provided context.
Use the context as the sole source of truth. Do not guess or use prior knowledge.
Answer with factual statements supported by the context.
Every factual claim must include an inline citation formatted as [Title | Section] placed immediately after the clause it supports.
Citations must use titles and section labels exactly as they appear in the context headers; do not invent, shorten, or paraphrase them.
If only part of the question is supported, answer only that part and state that the remaining parts are not in the provided context; do not ask to search online.
If you cannot answer with exact [Title | Section] citations from the context, respond exactly with: "I do not know based on the provided context because the retrieved sections do not mention this. Would you like me to find re

In [11]:
# Generate LLM answer
response_tier2 = rag_pipeline.run(query_tier2, k=TOP_K_RETRIEVAL, include_sources=True)

print("=" * 80)
print("LLM ANSWER")
print("=" * 80 + "\n")
print(response_tier2.answer)

print("\n" + "=" * 80)
print(f"SOURCES ({len(response_tier2.sources)} documents)")
print("=" * 80)
for i, source in enumerate(response_tier2.sources):
    print(f"\n[{i + 1}] {source.metadata.get('title', 'Unknown')}")
    print(f"    Section: {source.metadata.get('section', 'N/A')}")

2026-01-21 15:40:43,200 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


LLM ANSWER

The reward hyperparameters for DRL autofocus in the provided context are:

- Alpha (α): 100 [Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning | Reward function]
- Beta (β): 30 [Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning | Reward function]
- Mu (μ): 200 [Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning | Reward function]
- Delta (δ): 100 [Precision autofocus in optical microscopy with liquid lenses controlled by deep reinforcement learning | Reward function]

These hyperparameters incentivize the following aspects of the autofocus task:

- Alpha (α) encourages achieving clear imaging.
- Beta (β) rewards reducing the time to focus.
- Mu (μ) incentivizes stopping automatically once focused.
- Delta (δ) enhances the discriminative ability of the reward function by setting relatively large positive and negative re

**Comment:**
- The LLM answer is correct; the reward hyperparameters and their specific incentives are accurately identified.
- The answer is based on the relevant context provided in the text.
- The correct chunks were retrieved, namely Chunk 1 (Reward function) and Chunk 2 (Ablation experiments on the reward function).
- With the increased difficulty of the multi-detail question, the system still provides a useful answer.

### Tier 3: Synthesis / Cross-paper Question

In [12]:
# Tier 3 Query: Synthesis requiring reasoning across sources
query_tier3 = "How does FAST define 'scanning efficiency,' and in what way is this fundamentally different from raster-grid scanning?"

print(f"QUERY (Tier 3): {query_tier3}\n")
print("=" * 80)
print("RETRIEVAL RESULTS")
print("=" * 80)

# Retrieve chunks
query_embedding = embedder.encode([query_tier3])[0]
results = db_service.query(
    model_key=EMBEDDER_TYPE,
    query_embedding=query_embedding.tolist(),
    n_results=TOP_K_RETRIEVAL
)

log_retrieval_results(
    results=results,
    query=query_tier3,
    output_file=OUTPUT_DIR / "tier3_retrieval.txt"
)

QUERY (Tier 3): How does FAST define 'scanning efficiency,' and in what way is this fundamentally different from raster-grid scanning?

RETRIEVAL RESULTS

Rank 1 | Distance: 0.2472
ID:      Kandel_et_al.___2023___Demonstration_of_an_AI_driven_workflow_for_autonomous_high_resolution_scanning_microscopy.pdf#Discussion_part3
Section: Discussion
Paper:   Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy
Authors: Saugat Kandel, Tao Zhou, Anakha V. Babu, Zichao Di, Xinxin Li, Xuedan Ma, Martin Holt, Antonino Miceli, Charudatta Phatak, Mathew J. Cherukara

Content (1200 chars):
--------------------------------------------------------------------------------
As such, there could exist scenarios in which the time required for the motormovementeclipsesthe time required for a single measurement. We expect to address the latter challenge by explicitly including a measurement-density-based term 38 or a movement-time-based term in the candidate selection proce

"QUERY: How does FAST define 'scanning efficiency,' and in what way is this fundamentally different from raster-grid scanning?\n\n================================================================================\nRETRIEVAL RESULTS\n================================================================================\n\n\n================================================================================\nRank 1 | Distance: 0.2472\n================================================================================\nID:      Kandel_et_al.___2023___Demonstration_of_an_AI_driven_workflow_for_autonomous_high_resolution_scanning_microscopy.pdf#Discussion_part3\nSection: Discussion\nPaper:   Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy\nAuthors: Saugat Kandel, Tao Zhou, Anakha V. Babu, Zichao Di, Xinxin Li, Xuedan Ma, Martin Holt, Antonino Miceli, Charudatta Phatak, Mathew J. Cherukara\n\nContent (1200 chars):\n-------------------------------------------------

In [13]:
# Show exact prompt sent to LLM
show_llm_prompt(
    rag_pipeline=rag_pipeline,
    retriever=retriever,
    question=query_tier3,
    top_k=TOP_K_RETRIEVAL
)

EXACT PROMPT SENT TO LLM
Template: answer | Retrieved chunks: 5 | Context: 4945 chars


MESSAGE 1: SYSTEM

You are a RAG assistant answering questions about scientific PDFs using only the provided context.
Use the context as the sole source of truth. Do not guess or use prior knowledge.
Answer with factual statements supported by the context.
Every factual claim must include an inline citation formatted as [Title | Section] placed immediately after the clause it supports.
Citations must use titles and section labels exactly as they appear in the context headers; do not invent, shorten, or paraphrase them.
If only part of the question is supported, answer only that part and state that the remaining parts are not in the provided context; do not ask to search online.
If you cannot answer with exact [Title | Section] citations from the context, respond exactly with: "I do not know based on the provided context because the retrieved sections do not mention this. Would you like me to find re

In [14]:
# Generate LLM answer
response_tier3 = rag_pipeline.run(query_tier3, k=TOP_K_RETRIEVAL, include_sources=True)

print("=" * 80)
print("LLM ANSWER")
print("=" * 80 + "\n")
print(response_tier3.answer)

print("\n" + "=" * 80)
print(f"SOURCES ({len(response_tier3.sources)} documents)")
print("=" * 80)
for i, source in enumerate(response_tier3.sources):
    print(f"\n[{i + 1}] {source.metadata.get('title', 'Unknown')}")
    print(f"    Section: {source.metadata.get('section', 'N/A')}")

2026-01-21 15:40:50,367 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


LLM ANSWER

FAST defines 'scanning efficiency' as the ability to isolate regions of interest in sparse settings and prepare for pointwise scanning in these regions, or more generally, to guide any scanning microscopy experiment where full pointwise information is not needed [Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy | Discussion].

This definition is fundamentally different from raster-grid scanning because FAST does not require a systematic sampling of every point within the field of view. Instead, it strategically selects points based on estimated regions of interest or highest expected reduction in discrepancy (ERD) [Self-driving scanning microscopy workflow]. In contrast, raster-grid scanning systematically samples every point in a predefined grid pattern, regardless of whether those points are of interest or not.

SOURCES (5 documents)

[1] Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy
    

**Comment:**
- The LLM answer is PARTLY correct; it accurately describes the functionality of Fast Autonomous Scanning Toolkit (FAST) in isolating regions of interest (based on the paper "Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy")

- Failure: The LLM is missing crucial information. The chunks from a second paper that is necessary to answer the question in regards to raster-grid scanning ("Deep reinforcement learning for data‑driven adaptive scanning in ptychography"), which deals with raster-grid scanning, were not retrieved in the top 5 chunks passed to the LLM. The other paper retrieved ("A general Bayesian algorithm for the autonomous alignment of beamlines") is not directly relevant to answer the key aspects of the question.
    - Instead, since the paper "Demonstration of an AI-driven workflow for autonomous high-resolution scanning microscopy" does also mention raster-grid scanning briefly in the retrieved chunks, the LLM based its answer on the little information it could deduce from this *without alerting the user to missing relevant information*.
    - Therefore, the system ultimately fails in answering the main question of relating FAST to raster-grid scanning, mainly due to missing one of two relevant papers/chunks and also due to not alerting the user to a lack of information.

## 5. Systematic Evaluation

Evaluation across all questions in the dataset, measuring retrieval accuracy and answer quality.

In [15]:
# Load evaluation dataset
eval_dataset = load_eval_dataset()
# Enhanced RAG Evaluator with chunk-level, multi-paper, and answer quality metrics
evaluator = EnhancedRAGEvaluator(rag_pipeline)

2026-01-21 15:40:54,781 - INFO - Use pytorch device_name: cuda:0
2026-01-21 15:40:54,781 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Loaded 12 questions from D:\Dokumente\Studium\MSc\WS2526\GenAi\LiteraturAssistent\GenAI\eval_dataset.json
Loading semantic similarity model: all-MiniLM-L6-v2...
Model loaded successfully.


In [16]:
# Run evaluation
df_results = evaluator.evaluate(eval_dataset, top_k=5)

# Display summary
print("\n" + "=" * 80)
print("EVALUATION SUMMARY")
print("=" * 80 + "\n")

# Tier 1-2: Exact chunk matching
tier_12 = df_results[df_results['Tier'].isin([1, 2])]
if len(tier_12) > 0:
    chunk_match_rate = tier_12['Exact_Chunk_Match'].sum() / tier_12['Exact_Chunk_Match'].notna().sum()
    print(
        f"Tier 1-2 (Single Paper) - Exact Chunk Hit Rate: {chunk_match_rate:.2%} ({int(tier_12['Exact_Chunk_Match'].sum())}/{int(tier_12['Exact_Chunk_Match'].notna().sum())})")

    found_ranks = tier_12[tier_12['Exact_Chunk_Match'] == True]['Chunk_Rank']
    if len(found_ranks) > 0:
        print(f"  - Avg rank of correct chunk: {found_ranks.mean():.1f}")

    semantic_hits = tier_12[tier_12['Semantic_Chunk_Hit'] == True]
    if len(semantic_hits) > 0:
        print(f"  - Semantic near-miss hits: {len(semantic_hits)} (similarity > 0.7)")

    misses_with_sim = tier_12[(tier_12['Exact_Chunk_Match'] == False) & (tier_12['Best_Chunk_Similarity'].notna())]
    if len(misses_with_sim) > 0:
        print(f"  - Avg similarity for misses: {misses_with_sim['Best_Chunk_Similarity'].mean():.3f}")

# Tier 3: Multi-paper matching
tier_3 = df_results[df_results['Tier'] == 3]
if len(tier_3) > 0:
    multi_match_rate = tier_3['Multi_Paper_Match'].sum() / len(tier_3)
    print(
        f"\nTier 3 (Synthesis) - Multi-Paper Hit Rate: {multi_match_rate:.2%} ({int(tier_3['Multi_Paper_Match'].sum())}/{len(tier_3)})")
    print(f"  - Avg papers retrieved: {tier_3['Num_Papers'].mean():.1f}")

    tier_3_with_expected = tier_3[tier_3['Paper_Recall'].notna()]
    if len(tier_3_with_expected) > 0:
        print(f"  - Avg paper recall: {tier_3_with_expected['Paper_Recall'].mean():.2%}")
        print(f"  - Avg paper precision: {tier_3_with_expected['Paper_Precision'].mean():.2%}")

# Answer Quality
with_answer_eval = df_results[df_results['Answer_Similarity'].notna()]
if len(with_answer_eval) > 0:
    avg_answer_sim = with_answer_eval['Answer_Similarity'].mean()
    print(f"\nAnswer Quality (semantic similarity to expected):")
    print(f"  - Avg answer similarity: {avg_answer_sim:.3f} ({len(with_answer_eval)} questions)")
    print(f"  - High quality (>0.7): {(with_answer_eval['Answer_Similarity'] > 0.7).sum()}/{len(with_answer_eval)}")

print(f"\nAverage Latency: {df_results['Latency'].mean():.2f}s")

Starting evaluation of 12 questions...


  0%|          | 0/12 [00:00<?, ?it/s]2026-01-21 15:40:57,782 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:02<00:25,  2.30s/it]2026-01-21 15:40:59,797 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


...

 92%|█████████▏| 11/12 [00:40<00:03,  3.50s/it]2026-01-21 15:41:37,925 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:49<00:00,  4.13s/it]


EVALUATION SUMMARY

Tier 1-2 (Single Paper) - Exact Chunk Hit Rate: 100.00% (12/12)
  - Avg rank of correct chunk: 1.0

Answer Quality (semantic similarity to expected):
  - Avg answer similarity: 0.451 (12 questions)
  - High quality (>0.7): 0/12

Average Latency: 4.11s


In [17]:
# Detailed results table
print("=" * 80)
print("DETAILED RESULTS")
print("=" * 80)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 40)

print(df_results.to_string(index=False))

# Save results
output_filename = OUTPUT_DIR / "evaluation_results.csv"
df_results.to_csv(output_filename, index=False)
print(f"\nResults saved to {output_filename}")

DETAILED RESULTS
 Tier                                                        Question    Target_Tag  Exact_Chunk_Match  Chunk_Rank Semantic_Chunk_Hit Best_Chunk_Similarity  Num_Papers Multi_Paper_Match  Paper_Recall  Paper_Precision  Answer_Similarity                              Papers  Latency
    1 What physical quantity is the controller changing (the actua... liquid lenses               True           1               None                  None           2              None           1.0            0.500              0.253       Zhang et al. | Rebuffi et al.     2.27
    1 Which classic search methods are used as baselines in the DR...     autofocus               True           1               None                  None           2              None           1.0            0.500              0.458       Zhang et al. | Rebuffi et al.     2.96
    1 What is the main objective of 'adaptive scanning' compared t...  ptychography               True           1               None       

# Functionality 2: External Paper Search

## 1. External Paper Retrieval Demonstration

In [18]:
USER_QUERY = "How are LLMs used in plant growing?"
# function can be found at backend/utils.py
# executes standard RAG as see above but if search_for_new_context flag is TRUE, searches for new papers via Semantic Scholar
response = query_rag(
    rag_pipeline=rag_pipeline,
    retriever=retriever,
    rec_service=rec_service,
    question=USER_QUERY,
    search_for_new_context=True,
    top_k_results=3
)


Query: How are LLMs used in plant growing?

Retrieved 5 chunks



2026-01-21 15:41:46,716 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


ANSWER

I do not know based on the provided context because the retrieved sections do not mention this. Would you like me to find related papers online?

SOURCES

[1] AutoFocus: AI-driven alignment of nanofocusing X-ray mirror ...
    Section: 5. The AI-driven controller in operating conditions

[2] Self-Driving Laboratories for Chemistry and Materials Scienc...
    Section: 7. OPTOELECTRONICS

[3] Self-Driving Laboratories for Chemistry and Materials Scienc...
    Section: 8. ENERGY STORAGE MATERIALS

[4] Self-Driving Laboratories for Chemistry and Materials Scienc...
    Section: 4.7. Solid State Materials Synthesis

[5] An autonomous laboratory for the accelerated synthesis of no...
    Section: An autonomous laboratory for the accelerated synthesis of novel materials

DEBUG: Triggering online search for 3 papers...
Searching papers for: 'How are LLMs used in plant growing?'



2026-01-21 15:41:48,237 - INFO - HTTP Request: GET https://api.semanticscholar.org/graph/v1/paper/search?query=How+are+LLMs+used+in+plant+growing%3F&limit=3&fields=paperId%2Ctitle%2Cyear%2Curl%2Cauthors%2Cabstract "HTTP/1.1 200 OK"


NEW ANSWER (Enhanced with Online Search)

Found 3 online sources for context:

[Online Source 1]
Growth Promotion and Secondary Metabolites of Vegetables by Spraying Soil with Psidium guajava, Aloe vera, Allium sativum and Medicago sativa Extracts at Various Stages of Growth (2025)
Link: https://www.semanticscholar.org/paper/e9f6e956ce0384ee2bbd053ea9455e5c29f08f8b
Abstract: There is a growing need for sustainable, efficient methods to promote plant growth and protect crops, with plant extracts offering natural, multi-component solutions. Based on previous observations, Psidium guajava, Aloe vera, Allium sativum and Medicago sativa were selected from 17 water extracts to investigate how the application times of soil sprays affect the antioxidant enzymes and secondary metabolites in fruity and leafy vegetables at different growth stages. From 1 week after sowing (WAS...

[Online Source 2]
How to Leverage Agentic AI and Knowledge Graphs to Enhance Overall Equipment Efficiency (OEE) (2025

## 2. Paper Recommendations based on User Query

In [20]:
USER_QUERY = "List the reward hyperparameters (e.g., alpha, beta, mu, delta) for DRL autofocus and what each incentivizes."
negative_paper_ids = []
NEW_PAPER_LIMIT = 5

# get relevant papers to get new context
relevant_docs = retriever.get_relevant_documents(
    query=USER_QUERY,
    k=TOP_K_RETRIEVAL
)

# use relevant docs to get new paper recommendations
recommendations = asyncio.run(
    rec_service.get_recommendations_from_docs(
        relevant_docs=relevant_docs,
        negative_ids=negative_paper_ids,
        limit=NEW_PAPER_LIMIT
    )
)
# display results
results_text = [f"Found {len(recommendations)} online sources:\n"]
for i, paper in enumerate(recommendations, 1):
    title = paper.get("title", "Unknown Title")
    year = paper.get("year", "N/A")
    url = paper.get("url", "No URL available")
    abstract = paper.get("abstract") or "No abstract available."
    if len(abstract) > 500:
        abstract = abstract[:500] + "..."
    entry = (
        f"{'=' * 80}\n"
        f"[Online Source {i}]\n"
        f"{'=' * 80}\n"
        f"{title} ({year})\n"
        f"Link: {url}\n"
        f"Abstract: {abstract}\n"
        f"{'=' * 80}\n"
    )
    results_text.append(entry)

full_results = "\n".join(results_text)
print(full_results)

Extracted 2 unique titles from docs.


2026-01-21 15:42:00,008 - INFO - HTTP Request: GET https://api.semanticscholar.org/graph/v1/paper/search?query=AutoFocus%3A+AI-driven+alignment+of+nanofocusing+X-ray+mirror+systems&limit=1&fields=paperId "HTTP/1.1 200 OK"
2026-01-21 15:42:01,957 - INFO - HTTP Request: GET https://api.semanticscholar.org/graph/v1/paper/search?query=Precision+autofocus+in+optical+microscopy+with+liquid+lenses+controlled+by+deep+reinforcement+learning&limit=1&fields=paperId "HTTP/1.1 200 OK"


Requesting recommendations based on 2 papers...


2026-01-21 15:42:05,000 - INFO - HTTP Request: POST https://api.semanticscholar.org/recommendations/v1/papers/?fields=paperId%2Ctitle%2Cyear%2Curl%2Cauthors%2Cabstract&limit=5 "HTTP/1.1 200 OK"


Found 5 online sources:

[Online Source 1]
Standard design and evaluation of monolithic Wolter mirror for x-ray focusing at SPring-8. (2025)
Link: https://www.semanticscholar.org/paper/3a870b2a99db7f7a5e86cbd118c6024b658a8fc8
Abstract: This paper describes a monolithic Wolter mirror, a standard micron-focusing element at SPring-8, which achieves highly stable and efficient micro-focusing. The mirror is composed of an ellipsoidal and a hyperboloidal surface on a single substrate in a Wolter type-I configuration. This configuration forms a focusing optical system that is achromatic, highly stable, and readily adjustable. An optical design methodology was developed, taking into consideration beamline spatial constraints, energy-d...

[Online Source 2]
Optimizing the Focusing Performance of Diffractive Optical Elements by Integrated Structure Techniques and Laser Lithography (2026)
Link: https://www.semanticscholar.org/paper/3e486b37054fa8506067bc2a92283437aa5aa00c
Abstract: Diffractive op